In [53]:
import tensorflow as tf

In [54]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


In [55]:
learning_rate = 0.001
training_epochs = 20
batch_size = 128  
display_step = 1

n_input = 784 
n_classes = 10 
n_hidden_layer = 256

In [56]:
weights = {'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])), \
          'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes])) }

biases = {'hidden_layer': tf.Variable(tf.zeros_like([n_hidden_layer], dtype = 'float32')), \
          'out': tf.Variable(tf.zeros_like([n_classes], dtype = 'float32'), name='bias_out') }

In [57]:
x = tf.placeholder("float", [None, 28, 28, 1])
y = tf.placeholder("float", [None, n_classes])

x_flat = tf.reshape(x, [-1, n_input])

In [58]:
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)

out = tf.add(tf.matmul(layer_1, weights['out']), biases['out'])

In [59]:
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)
    
correct_prediction = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [60]:
init = tf.global_variables_initializer()

save_file = './model.ckpt'
saver = tf.train.Saver()    

In [61]:
with tf.Session() as session:
    session.run(init)
    
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            session.run(optimizer, feed_dict={x: batch_x, y: batch_y})
            
        if epoch % 5 == 0:
            valid_accuracy = session.run(accuracy, \
                feed_dict={ \
                    x: mnist.validation.images, \
                    y: mnist.validation.labels}) 
            print('Epoch {:<3} - Validation Accuracy: {}'.format( \
                epoch, \
                valid_accuracy)) \
    
    saver.save(session, save_file)
    
    print('Weights_hidden:')
    print(session.run(weights['hidden_layer']))
    print('Bias_hidden:')
    print(session.run(biases['hidden_layer']))

Epoch 0   - Validation Accuracy: 0.3330000042915344
Epoch 5   - Validation Accuracy: 0.6815999746322632
Epoch 10  - Validation Accuracy: 0.7462000250816345
Epoch 15  - Validation Accuracy: 0.77920001745224
Weights_hidden:
[[-0.71542341  0.81547058  0.40509087 ..., -1.40644395  0.72746336
  -1.21636331]
 [-0.67631149  0.39498919  0.4927645  ..., -0.70587647 -1.82230079
   0.12502505]
 [-1.23066986  0.50444639  0.49351573 ..., -2.0194416  -0.26989356
   0.30743435]
 ..., 
 [ 0.41826931  0.33792326 -0.95405525 ...,  0.32898     0.46050224
  -0.9028964 ]
 [-0.20934014  0.42786774  0.7566843  ..., -1.16201508 -0.61248893
   0.29206827]
 [-1.71332717 -0.74021012  0.42013714 ..., -1.4052732  -0.59926176
  -0.59334719]]
Bias_hidden:
[-5.3154254]


In [62]:
##tf.reset_default_graph()

In [63]:
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(
        accuracy,
        feed_dict={x: mnist.test.images, y: mnist.test.labels})

print('Test Accuracy: {}'.format(test_accuracy))

Test Accuracy: 0.802299976348877


In [64]:
print('Accuracy: {}'.format(accuracy.name))
print('Hiiden input name: {}'.format(weights['hidden_layer'].name))
print('Biases out name: {}'.format(biases['out'].name))

Accuracy: Mean_3:0
Hiiden input name: Variable_7:0
Biases out name: bias_out_1:0
